## Model:QA Calls For Evaluation
Logging every call to the chain for later review/analysis

In [1]:
%pip install langchain langsmith --quiet
%pip cache purge

Files removed: 0


In [2]:
%pip install optimum --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.5 MB/s eta 0:00:00


In [4]:
!pip install langchain_community --quiet
!pip install langchain_openai --quiet
!pip install huggingface_hub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.0 MB/s eta 0:00:00


In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from dotenv import load_dotenv
import os
import openai

 ## Define Pipelines

In [6]:
from transformers import pipeline

hf_pipes = {
    "distilgpt2": pipeline("text-generation", model="distilgpt2", max_new_tokens=64),
    "flan-t5-base": pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=64),
    "flan-t5-large": pipeline("text2text-generation", model="google/flan-t5-large", max_new_tokens=64),
}

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [17]:
import requests
from google.colab import userdata
TOGETHER_API_KEY = userdata.get('TOGETHER_TOKEN')  # <-- paste your key

def together_mistral_call(prompt, max_tokens=128, temp=0.0):
    url = "https://api.together.ai/models/mistralai/Mistral-7B-Instruct-v0.2"
    headers = {"Authorization": f"Bearer {TOGETHER_API_KEY}"}
    payload = {
        "model": "mistralai/Mistral-7B-Instruct-v0.2",  # or try "mistralai/Mixtral-8x7B-Instruct-v0.1"!
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temp,
        "stop": None
    }
    response = requests.post(url, headers=headers, json=payload)
    try:
        result = response.json()
        # Structure: {'id': ..., 'object': ..., 'choices': [{'text': ...}], ...}
        if "choices" in result and result["choices"]:
            return result["choices"][0]["text"].strip()
        elif "error" in result:
            return f"API error: {result['error']}"
        else:
            return str(result)
    except Exception as e:
        return f"Non-JSON response: {response.text[:200]}"

In [ ]:
import openai

def openai_gpt35_call(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=64
    )
    return response.choices[0].message.content.strip()

In [ ]:
whisper_asr = pipeline("automatic-speech-recognition", model="openai/whisper-base")

In [18]:
test_set = [
    {"question": "What is ServiceNow?", "gold_answer": "ServiceNow is a cloud platform for digital workflows."},
    {"question": "Explain ITSM.", "gold_answer": "ITSM stands for IT Service Management."},
    # Add more samples...
]

## Evaluation Loop

In [ ]:
from tqdm import tqdm

results = []

for example in tqdm(test_set):
    q = example["question"]
    gold = example["gold_answer"]
    row = {"question": q, "gold_answer": gold}

    # Hugging Face local models (e.g., distilgpt2, flan-t5-base, flan-t5-large)
    for name, pipe in hf_pipes.items():
        if 'text2text' in pipe.task:
            try:
                out = pipe(q)[0]['generated_text']
            except Exception as e:
                out = str(e)
        else:
            try:
                out = pipe(q)[0]['generated_text']
            except Exception as e:
                out = str(e)
        row[name] = out

    # Mistral-7B via Together.ai
    try:
        row["mistral-7b"] = together_mistral_call(q)
    except Exception as e:
        row["mistral-7b"] = f"Together.ai error: {e}"

    # OpenAI GPT-3.5-turbo
    try:
        row["gpt-3.5-turbo"] = openai_gpt35_call(q)
    except Exception as e:
        row["gpt-3.5-turbo"] = f"OpenAI error: {e}"

    results.append(row)

In [ ]:
for example in audio_test_set:
    audio_path = example['audio_file']
    gold = example['gold_transcript']
    asr_out = whisper_asr(audio_path)['text']

## Calculate Metrics

In [ ]:
from sklearn.metrics import f1_score
def token_f1(a, b):
    a, b = set(a.lower().split()), set(b.lower().split())
    if not a or not b: return 0
    common = a & b
    precision = len(common)/len(a)
    recall = len(common)/len(b)
    return 2*precision*recall/(precision+recall+1e-8)

for r in results:
    for model in hf_pipes.keys() + ["gpt-3.5-turbo"]:
        r[f'{model}_f1'] = token_f1(r[model], r['gold_answer'])


## Compare Results

In [ ]:
from sklearn.metrics import f1_score

def token_f1(a, b):
    a, b = set(a.lower().split()), set(b.lower().split())
    if not a or not b: return 0.0
    common = a & b
    precision = len(common)/len(a)
    recall = len(common)/len(b)
    return 2*precision*recall/(precision+recall+1e-8)

import pandas as pd
df = pd.DataFrame(results)

for model in ["distilgpt2", "flan-t5-base", "flan-t5-large", "mistral-7b", "gpt-3.5-turbo"]:
    df[f"{model}_f1"] = df.apply(lambda r: token_f1(str(r[model]), str(r["gold_answer"])), axis=1)

print(df[[f"{model}_f1" for model in ["distilgpt2", "flan-t5-base", "flan-t5-large", "mistral-7b", "gpt-3.5-turbo"]]].mean())


In [ ]:
import matplotlib.pyplot as plt

means = df[[f"{model}_f1" for model in ["distilgpt2", "flan-t5-base", "flan-t5-large", "mistral-7b", "gpt-3.5-turbo"]]].mean()
means.plot(kind="bar", title="Average F1 Score per Model")
plt.ylabel("Token-level F1")
plt.show()

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("all_model_eval_results.csv", index=False)
df.head()